In [1]:
%%capture
!pip install import_ipynb --no-cache
import import_ipynb

In [2]:
%%capture
m = __import__("Methods")

# Feature Voting
- Weighs the importance of features based on the frequency between models, Shapley values, and lowest RMSE

In [3]:
import pandas as pd
import numpy as np
import ast
from collections import defaultdict
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 60)

### Dataset Cleaning

In [4]:
df = pd.read_csv("data/results/model_results_n3000.csv")

In [5]:
df

,Model,Target Variable,MSE,RMSE,MAE,R2,R,Selected Features,Sorted Shap Values
0,ANN_Final_Test,th_positive_cells,2652.076971,51.498320,28.483265,-0.809668,NaN,"7SLRNA:CR32864,abd-A,Abd-B,Abl,abo,Ace,acj6,nAChRalpha1,...","{'Fbp1': 0.6175863081031662, 'gammaTry': 0.1568093957583..."
1,Linear_Regression_Final_Test,th_positive_cells,2042.892604,45.198369,25.071024,-0.393986,NaN,"Cp38,cv-2,Dab,Dfd,disco,Delta,bsh,ea,Edg78E,Est-P,Taf9,e...","{'CG3819': 0.07599323966654625, 'Dfd': 0.055372169286853..."
2,Ridge_Final_Test,th_positive_cells,2030.433041,45.060327,24.990324,-0.385484,NaN,"Fbp1,CG3819,GATAe,CG34002,CG5897,asRNA:CR44106,lncRNA:CR...","{'Fbp1': 0.03646496059075438, 'CG3819': 0.03262355306834..."
3,Lasso_Final_Test,th_positive_cells,1986.779792,44.573308,24.732177,-0.355697,NaN,"7SLRNA:CR32864,abd-A,Abd-B,Abl,abo,Ace,acj6,nAChRalpha1,...","{'snk': 0.5626258972541195, 'lncRNA:CR45188': 0.18243525..."
4,SVM_Final_Test,th_positive_cells,2038.254701,45.147034,25.050882,-0.390821,NaN,"Abd-B,acj6,Adh,al,amd,Argk1,Arr1,Arr2,ase,btd,by,c(3)G,C...","{'Fbp1': 0.032755632660914955, 'CG3819': 0.0297635502707..."
...,...,...,...,...,...,...,...,...,...
7,Linear_Regression_Final_Test,repo_glial_cells,50699.446595,225.165376,104.042219,-0.274177,NaN,"Cp19,Cp36,cv-2,CycA,CycB,Cyt-b5-r,Dab,dec,Pkg21D,Dip-B,D...","{'CG7126': 0.03550376790631025, 'Cp36': 0.03255355884668..."
8,Ridge_Final_Test,repo_glial_cells,50730.989891,225.235410,104.083323,-0.274969,NaN,"CG7126,CG43996,CG1387,CG14260,CR30409,asRNA:CR45371,CR43...","{'CG7126': 0.031388015131363314, 'CG43996': 0.0213343584..."
9,Lasso_Final_Test,repo_glial_cells,50522.362003,224.771800,103.872901,-0.269726,NaN,"7SLRNA:CR32864,abd-A,Abd-B,Abl,abo,Ace,acj6,nAChRalpha1,...","{'ddbt': 0.43145103942503793, 'prd': 0.2957561177023954,..."
10,SVM_Final_Test,repo_glial_cells,50717.985908,225.206541,104.019568,-0.274643,NaN,"7SLRNA:CR32864,Abd-B,Act57B,Gart,Adh,Adhr,al,amd,Amy-p,a...","{'djl': 0.035060921352132, 'Tengl3': 0.03152701489366444..."


In [6]:
repo_df = df[df["Target Variable"] == "th_positive_cells"].copy()
repo_df.drop(columns=["Selected Features"], inplace=True)
repo_df

,Model,Target Variable,MSE,RMSE,MAE,R2,R,Sorted Shap Values
0,ANN_Final_Test,th_positive_cells,2652.076971,51.498320,28.483265,-0.809668,NaN,"{'Fbp1': 0.6175863081031662, 'gammaTry': 0.1568093957583..."
1,Linear_Regression_Final_Test,th_positive_cells,2042.892604,45.198369,25.071024,-0.393986,NaN,"{'CG3819': 0.07599323966654625, 'Dfd': 0.055372169286853..."
2,Ridge_Final_Test,th_positive_cells,2030.433041,45.060327,24.990324,-0.385484,NaN,"{'Fbp1': 0.03646496059075438, 'CG3819': 0.03262355306834..."
3,Lasso_Final_Test,th_positive_cells,1986.779792,44.573308,24.732177,-0.355697,NaN,"{'snk': 0.5626258972541195, 'lncRNA:CR45188': 0.18243525..."
4,SVM_Final_Test,th_positive_cells,2038.254701,45.147034,25.050882,-0.390821,NaN,"{'Fbp1': 0.032755632660914955, 'CG3819': 0.0297635502707..."
5,Random_Forest_Final_Test,th_positive_cells,2051.461866,45.293066,25.037196,-0.399833,NaN,"{'wbl': 0.00345565416948028, 'lncRNA:CR45809': 0.0034556..."


### RMSE & SHAP Summary Statistics

In [7]:
# Prepare lists for SHAP values and RMSE
shap_values_list = []
rmse_list = []

for index, row in repo_df.iterrows():
    # Convert SHAP values from string to actual dictionary
    shap_values = ast.literal_eval(row['Sorted Shap Values'])
    
    # Extend the list with the SHAP values
    shap_values_list.extend(shap_values.values())

    # Append the RMSE value
    rmse_list.append(row['RMSE'])

# Convert lists to pandas Series for statistical summary
shap_values_series = pd.Series(shap_values_list)
rmse_series = pd.Series(rmse_list)

# Display summary statistics
shap_stats = shap_values_series.describe()
rmse_stats = rmse_series.describe()

print("Summary Statistics for SHAP Values:")
print(shap_stats)
print("\nSummary Statistics for RMSE:")
print(rmse_stats)

Summary Statistics for SHAP Values:
count    18000.000000
mean         0.000733
std          0.006868
min          0.000000
25%          0.000000
50%          0.000055
75%          0.000364
max          0.617586
dtype: float64

Summary Statistics for RMSE:
count     6.000000
mean     46.128404
std       2.642762
min      44.573308
25%      45.082004
50%      45.172702
75%      45.269392
max      51.498320
dtype: float64


### Ensemble Feature Voting Implementation

In [15]:
from collections import defaultdict
import pandas as pd
import ast
import numpy as np

# Create dictionaries to store feature importance and frequency
feature_importance = defaultdict(float)
feature_frequency = defaultdict(float)

# Extract RMSE, R², and SHAP values
rmse_values = repo_df['RMSE'].values
r2_values = repo_df['R2'].values

# Normalize RMSE and R² using min-max scaling
rmse_min, rmse_max = min(rmse_values), max(rmse_values)
r2_min, r2_max = min(r2_values), max(r2_values)

repo_df['RMSE_scaled'] = (repo_df['RMSE'] - rmse_min) / (rmse_max - rmse_min)
repo_df['R2_scaled'] = (repo_df['R2'] - r2_min) / (r2_max - r2_min)

# Loop through the DataFrame and calculate importance
for index, row in repo_df.iterrows():
    shap_values = row['Sorted Shap Values']
    
    # Convert SHAP values from string to dictionary
    shap_values = ast.literal_eval(shap_values)
    
    # Normalize SHAP values (min-max scaling per model)
    shap_array = np.array(list(shap_values.values()))
    shap_min, shap_max = shap_array.min(), shap_array.max()
    
    if shap_max > shap_min:  # Avoid division by zero
        shap_values = {k: (v - shap_min) / (shap_max - shap_min) for k, v in shap_values.items()}
    
    # Scaled RMSE and R²
    rmse_scaled = row['RMSE_scaled']
    r2_scaled = row['R2_scaled']
    
    # Weight adjustment by (1 + R²_scaled) to emphasize models with higher R²
    weight_adjustment = (1 + r2_scaled)

    # SHAP importance scaling
    for feature, shap_value in shap_values.items():
        if shap_value != 0:
            feature_importance[feature] += shap_value * weight_adjustment
            feature_frequency[feature] += (1 / (rmse_scaled + 1e-6)) * weight_adjustment  # Avoid division by zero

# Adjust final importance by multiplying by the RMSE and R²-weighted frequency
for feature in feature_importance:
    feature_importance[feature] *= feature_frequency[feature]

# Sort features by their adjusted importance scores
sorted_features = sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)

# Create a DataFrame for the top 20 features
df_sorted_features = pd.DataFrame(sorted_features, columns=['Gene', 'Weight'])

# Display the top 20 important features
df_sorted_features.head(20)

df_sorted_features.to_csv("test test.csv", index=False)


In [16]:
df_sorted_features.head(20)

,Gene,Weight
0,snk,4.057546e+06
1,lncRNA:CR45188,1.341210e+06
2,CG46388,9.433750e+05
3,prd,8.014938e+05
4,Fbp1,4.158262e+02
5,CG3819,3.879623e+02
6,CG9411,2.349030e+02
7,CG34002,2.220437e+02
8,CR43281,1.961197e+02
9,CG5897,1.791819e+02


# Gene Set Analysis

In [17]:
# look at the names of the libraries that are available to use for PEA and GSEA
gene_sets = m.gp.get_library_name(organism="fly")

gene_sets.remove("Allele_Phenotypes_from_FlyBase_2017")

gene_list = df_sorted_features[:20]['Gene'].tolist()

In [18]:
# Perform GO enrichment analysis with FlyBase genes using Enrichr
enr = m.gp.enrichr(
    gene_list=gene_list,  # List of FlyBase gene symbols
    gene_sets=gene_sets,
    organism='fly',  # Set to 'fly' for Drosophila (FlyBase genes)
    outdir=None
)

enr.results[enr.results['Adjusted P-value'] < 0.05][['Term', 'Genes', 'Adjusted P-value']]

,Term,Genes,Adjusted P-value
0,subesophageal ganglion,Dfd,0.042423
1,embryonic mandibular segment,Dfd,0.042423
2,intestinal stem cell of posterior adult midgut epithelium,GATAe,0.042423
3,embryonic maxillary segment,Dfd,0.042423
4,vibrissae,Dfd,0.042423
5,lateral process,Dfd,0.042423
6,hypostomal sclerite,Dfd,0.044523
7,pair rule phenotype,prd,0.042423
8,poor,Dfd,0.047547
9,wild-type,prd,0.047547


In [19]:
results = enr.results[enr.results['Adjusted P-value'] < 0.05][['Term', 'Genes', 'Adjusted P-value']].sort_values(by=['Adjusted P-value'], ascending=False)

In [20]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
results

,Term,Genes,Adjusted P-value
11,mouth hooks,Dfd,0.049604
795,CG7548,CG9411,0.049294
800,CG12902,prd,0.049294
798,Cpr49Af,CG9411,0.049294
801,CG10898,prd,0.049294
776,CG14400,prd,0.049294
777,CG15027,prd,0.049294
784,CG9527,Dfd,0.049294
785,CG13390,prd,0.049294
770,Cpr100A,CG9411,0.049294


# Pathway Analysis Prep

In [21]:
symbols = df_sorted_features["Gene"].to_list()

import pandas as pd

# Load the conversion DataFrame
conversion_df = pd.read_csv("data/datasets/conversion_df.csv")

# Assuming the columns in conversion_df are named 'GeneSymbol' and 'FlyBaseID'
# If they have different names, adjust accordingly.
conversion_dict = dict(zip(conversion_df['Gene Symbol'], conversion_df['FB ID']))

# Assuming `symbols` contains the gene symbols
flybase_ids = [conversion_dict.get(symbol) for symbol in symbols]

# Now `flybase_ids` contains the FlyBase IDs corresponding to the gene symbols.
flybase_ids[:20]

['FBgn0003450',
 'FBgn0266698',
 'FBgn0286834',
 'FBgn0003145',
 'FBgn0000639',
 'FBgn0036833',
 'FBgn0030569',
 'FBgn0054002',
 'FBgn0262970',
 'FBgn0036220',
 'FBgn0263654',
 'FBgn0264915',
 'FBgn0265982',
 'FBgn0034563',
 'FBgn0047092',
 'FBgn0028518',
 'FBgn0000439',
 'FBgn0029966',
 'FBgn0038391',
 'FBgn0266628']

In [22]:
# Specify the output file name
output_file = "background_genes.txt"

# Write the FlyBase IDs to the text file
with open(output_file, 'w') as file:
    for fb_id in conversion_df["FB ID"]:
        file.write(f"{fb_id}\n")

print(f"FlyBase IDs have been written to {output_file}")


FlyBase IDs have been written to background_genes.txt


In [23]:
# Specify the output file name
output_file = "th_selected_genes.txt"

# Write the FlyBase IDs to the text file
with open(output_file, 'w') as file:
    for fb_id in flybase_ids:
        file.write(f"{fb_id}\n")

print(f"FlyBase IDs have been written to {output_file}")


FlyBase IDs have been written to th_selected_genes.txt
